# All American Transfers

In [1]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage

## Build out DataFrames

In [2]:
SQL = '''SELECT * from Transfers247'''
df_247 = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,StandardizedPosition,KeyPositionGroup
247_IDYR,,,,,
paulthurston_nebraska_2012,paulthurston_nebraska,paulthurston,2012,OL,1
gregmcmullen_nebraska_2012,gregmcmullen_nebraska,gregmcmullen,2012,DE,2
jordanwesterkamp_nebraska_2012,jordanwesterkamp_nebraska,jordanwesterkamp,2012,WR,1
imanicross_nebraska_2012,imanicross_nebraska,imanicross,2012,RB,1
tommyarmstrong_nebraska_2012,tommyarmstrong_nebraska,tommyarmstrong,2012,QB,1
...,...,...,...,...,...
dondrellharris_troy_2013,dondrellharris_troy,dondrellharris,2013,QB,1
jordanchunn_troy_2013,jordanchunn_troy,jordanchunn,2013,FB,1
bryanslater_troy_2013,bryanslater_troy,bryanslater,2013,DE,2


In [8]:
SQL = '''SELECT ID, PlayerName, Year from UnlinkedAllAmerican'''
df_aa = (fx.connDBAndReturnDF(SQL)).set_index('ID')
df_aa.index.name = 'AllAmerican_ID'
df_aa

,PlayerName,Year
AllAmerican_ID,,
aaroncurry_wakeforest,aaroncurry,2008
adampacmanjones_westvirginia,adampacmanjones,2004
adamsnyder_oregon,adamsnyder,2004
aidanschneider_oregon,aidanschneider,2015
alanzemaitis_pennstate,alanzemaitis,2005
...,...,...
willsmith_ohiostate,willsmith,2003
willsnyderwine_duke,willsnyderwine,2010
zachmiller_arizonastate,zachmiller,2006


### NOTES

> you don't

In [9]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['PlayerName'])

In [10]:
pairs = indexer.index(df_aa, df_247)

In [11]:
from functions import YearNFL

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
#c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
#sumFields.append('StandardizedPosition')
#c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
#sumFields.append('KeyPositionGroup')
c.add(YearNFL('Year', 'Year', label='Year'))
sumFields.append('Year')

In [13]:
features = c.compute(pairs, df_aa, df_247)

In [14]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['Sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

,,PlayerName,Year,Sum
AllAmerican_ID,247_IDYR,,,
andyisabella_umass,andyisabella_massachusetts_2015,1.0,1.0,1.0
garrettbradbury_ncstate,garrettbradbury_northcarolinastate_2014,1.0,1.0,1.0
harrisonbryant_fau,harrisonbryant_floridaatlantic_2016,1.0,1.0,1.0
jaelanphillips_miami,jaelanphillips_ucla_2017,1.0,1.0,1.0
jeremycash_duke,jeremycash_ohiostate_2011,1.0,1.0,1.0
joeburrow_lsu,joeburrow_ohiostate_2015,1.0,1.0,1.0
jonathangreenard_florida,jonathangreenard_louisville_2015,1.0,1.0,1.0
joseborregales_miami,joseborregales_floridainternational_2016,1.0,1.0,1.0
jrreed_georgia,jrreed_tulsa_2014,1.0,1.0,1.0


In [15]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    if (data['Sum'] > .94):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsAllAmericanTransfer.csv")

In [ ]:
dfFinal

In [17]:
fuzzyMI = pd.MultiIndex.from_frame(dfFinal)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_AATransfers.json",
    fuzzyMI[0:200],
    df_aa,
    df_247,
    dataset_a_name="All American",
    dataset_b_name="Master"
)

In [18]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result_AATransfers.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

In [19]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 4, 1, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()